In [ ]:
import pandas as pd

In [ ]:
currencies = {
    "PLN": "PLN",
    "EUR": "€",
    "GBP": "£",
    "USD": "$"
}

In [ ]:
choosen_currency = "EUR"

In [ ]:
currency = currencies[choosen_currency]

In [ ]:
file_csv = "E979494D-7905-49C7-970D-5B069B09CDEA.csv"

In [ ]:
df = pd.read_csv(file_csv)

In [ ]:
print(df.head())

In [ ]:
if choosen_currency != 'USD':
    df = df[df['Price'].str.contains(currency) | df['Value'].str.contains('currency') | df['Fees'].str.contains(currency)]
else:
    df = df[df['Price'].str.contains(r'\$', regex=True) | 
        df['Value'].str.contains(r'\$', regex=True) | 
        df['Fees'].str.contains(r'\$', regex=True)]

In [ ]:
df.head()

In [ ]:

df['Price'] = df['Price'].str.replace(currency, '', regex=True).str.replace(',', '.', regex=False).str.replace(' ', '', regex=False).str.replace(r'\s+', '', regex=True).astype(float)

df['Value'] = df['Value'].str.replace(currency, '', regex=True).str.replace(',', '.', regex=False).str.replace(' ', '', regex=False).str.replace(r'\s+', '', regex=True).astype(float)

df['Fees'] = df['Fees'].str.replace(currency, '', regex=True).str.replace(',', '.', regex=False).str.replace(' ', '', regex=False).str.replace(r'\s+', '', regex=True).astype(float)


In [ ]:
df.head()

In [ ]:
month_translation = {
    "sty": "Jan", "lut": "Feb", "mar": "Mar", "kwi": "Apr", "maj": "May", "cze": "Jun",
    "lip": "Jul", "sie": "Aug", "wrz": "Sep", "paź": "Oct", "lis": "Nov", "gru": "Dec"
}

In [ ]:
def translate_month(date_str):
    for pl_month, en_month in month_translation.items():
        if pl_month in date_str:
            return date_str.replace(pl_month, en_month)
    return date_str

In [ ]:
df['Date'] = df['Date'].apply(translate_month)

df['Date'] = pd.to_datetime(df['Date'], format='%d %b %Y, %H:%M:%S')

df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

In [ ]:
from datetime import datetime, timedelta
import requests

def get_rate_for_date(date_str, code, max_days_forward=5, rate_type="ask"):
    """
    Pobiera kurs z NBP dla danej daty.
    Jeśli brak danych, próbuje kolejnych dni (do max_days_forward).
    """
    base_date = datetime.strptime(date_str, "%Y-%m-%d")
    
    for i in range(max_days_forward + 1):
        check_date = (base_date + timedelta(days=i)).strftime("%Y-%m-%d")
        url = f"https://api.nbp.pl/api/exchangerates/tables/C/{check_date}"
        
        try:
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                for rate in data[0]['rates']:
                    if rate['code'] == code:
                        return rate[rate_type]
        except:
            continue

    print(f"Brak kursu ask dla {date_str} w ciągu {max_days_forward} dni.")
    return None

In [ ]:
if choosen_currency == "PLN":
    purchase_total = df[df['Type'] == 'Kupno']['Value'].sum()
    purchase_fees_total = df[df['Type'] == 'Kupno']['Fees'].sum()
    sale_total = df[df['Type'] == 'Sprzedaż']['Value'].sum()
    sale_fees_total = df[df['Type'] == 'Sprzedaż']['Fees'].sum()
    
    print(f"\nSuma wartości kupna w PLN: {purchase_total:.2f}")
    print(f"Suma prowizji kupna w PLN: {purchase_fees_total:.2f}")
        
    print(f"\nSuma wartości sprzedaży w PLN: {sale_total:.2f}")
    print(f"Suma prowizji sprzedaży w PLN: {sale_fees_total:.2f}")

else:
    purchase_info = df[df['Type'] == 'Kupno'][['Value', 'Fees', 'Date']]
    sale_info = df[df['Type'] == 'Sprzedaż'][['Value', 'Fees', 'Date']]
    unique_dates = purchase_info['Date'].unique()
    ask_rates = {}

    for date in unique_dates:
        ask_rate = get_rate_for_date(date, choosen_currency)
        ask_rates[date] = ask_rate

    purchase_info['AskRate'] = purchase_info['Date'].map(ask_rates)

    purchase_info['ValuePLN'] = purchase_info['Value'] * purchase_info['AskRate']
    purchase_info['FeesPLN'] = purchase_info['Fees'] * purchase_info['AskRate']

    total_value_pln = purchase_info['ValuePLN'].sum()
    total_fees_pln = purchase_info['FeesPLN'].sum()

    print(purchase_info[['Date', 'Value', 'AskRate', 'ValuePLN', 'FeesPLN']])
    print(f"\nSuma wartości w PLN: {total_value_pln:.2f}")
    print(f"Suma prowizji w PLN: {total_fees_pln:.2f}")